### 1. pandas 및 pickle 불러오기

In [17]:
import pandas as pd
import os

num_to_label pickle 에서 불러오려면 아래 셀 uncomment 후 경로 지정

In [2]:
# import pickle
# with open("../code_baseline/dict_num_to_label.pkl",'rb') as fr:
#     num_to_label = pickle.load(fr)
# print(num_to_label)

num_to_label dict 직접 만들기

In [3]:
num_to_label = {0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}

### 2. 앙상블 재료들 DF로 불러오기

In [4]:
# 본인이 앙상블할 재료 submission.csv들로 경로 변경
ens1_name= 'train_21_roberta-large'
ens2_name = 'train_19_roberta-large_lr'
ens1 = pd.read_csv(f'./ensemble/{ens1_name}.csv')
ens2 = pd.read_csv(f'./ensemble/{ens2_name}.csv')

In [5]:
# 잘 import 되었나 길이 확인
print(len(ens1))
print(len(ens2))

7765
7765


두 모델이 다르게 예측한 pred_label의 개수 확인


In [6]:
result_comparison = ens1['pred_label'] != ens2['pred_label']
count_true = result_comparison.sum()
print(f"다른 예측 개수 : {count_true}")

다른 예측 개수 : 719


### 3. 확률 계산을 위해 'probs' 칼럼 float 리스트로 만들어서 새 칼럼에 추가

In [7]:
ens1['probs_float'] = ens1['probs'].apply(lambda x: list(map(float, x[1:-1].split(', '))))
ens2['probs_float'] = ens2['probs'].apply(lambda x: list(map(float, x[1:-1].split(', '))))

In [8]:
# probs_float 칼럼이 잘 생겼음을 확인
ens1.head()

,id,pred_label,probs,probs_float
0,0,org:product,"[0.03520144522190094, 0.002557179192081094, 0....","[0.03520144522190094, 0.002557179192081094, 0...."
1,1,per:alternate_names,"[0.019070588052272797, 0.00022892268316354603,...","[0.019070588052272797, 0.00022892268316354603,..."
2,2,no_relation,"[0.9993014335632324, 2.695106013561599e-05, 4....","[0.9993014335632324, 2.695106013561599e-05, 4...."
3,3,no_relation,"[0.9800989627838135, 9.629613487049937e-05, 0....","[0.9800989627838135, 9.629613487049937e-05, 0...."
4,4,no_relation,"[0.766914963722229, 0.22684572637081146, 0.000...","[0.766914963722229, 0.22684572637081146, 0.000..."


In [9]:
ens2.head()

,id,pred_label,probs,probs_float
0,0,org:members,"[0.01552027091383934, 0.0003683916584122926, 0...","[0.01552027091383934, 0.0003683916584122926, 0..."
1,1,per:title,"[0.004669270012527704, 6.69287983328104e-05, 0...","[0.004669270012527704, 6.69287983328104e-05, 0..."
2,2,no_relation,"[0.9996310472488403, 1.7726570149534382e-05, 1...","[0.9996310472488403, 1.7726570149534382e-05, 1..."
3,3,no_relation,"[0.9900015592575073, 5.1353988965274766e-05, 6...","[0.9900015592575073, 5.1353988965274766e-05, 6..."
4,4,no_relation,"[0.9947687387466431, 0.004142543766647577, 5.0...","[0.9947687387466431, 0.004142543766647577, 5.0..."


### 4. copy를 통해 앙상블 출력용 DataFrame의 뼈대 만들기

In [10]:
ens_result = ens1.copy()
ens_result.head()

,id,pred_label,probs,probs_float
0,0,org:product,"[0.03520144522190094, 0.002557179192081094, 0....","[0.03520144522190094, 0.002557179192081094, 0...."
1,1,per:alternate_names,"[0.019070588052272797, 0.00022892268316354603,...","[0.019070588052272797, 0.00022892268316354603,..."
2,2,no_relation,"[0.9993014335632324, 2.695106013561599e-05, 4....","[0.9993014335632324, 2.695106013561599e-05, 4...."
3,3,no_relation,"[0.9800989627838135, 9.629613487049937e-05, 0....","[0.9800989627838135, 9.629613487049937e-05, 0...."
4,4,no_relation,"[0.766914963722229, 0.22684572637081146, 0.000...","[0.766914963722229, 0.22684572637081146, 0.000..."


### 5. 앙상블 재료들의 확률끼리 numpy 이용해서 평균 산출

In [11]:
import numpy as np

# 실행 시 오류 날텐데 무시 가능..
for i in range(len(ens_result)):
    ens_result['probs_float'][i] = np.mean([ens1['probs_float'][i], ens2['probs_float'][i]], axis=0)

<ipython-input-11-9e78e1991d9c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_result['probs_float'][i] = np.mean([ens1['probs_float'][i], ens2['probs_float'][i]], axis=0)
<ipython-input-11-9e78e1991d9c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_result['probs_float'][i] = np.mean([ens1['probs_float'][i], ens2['probs_float'][i]], axis=0)
<ipython-input-11-9e78e1991d9c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en

기존 probs 칼럼도 평균된 probs_float 값으로 바꿔주기

In [12]:
ens_result['probs'] = ens_result['probs_float'].apply(lambda x: '[' + ', '.join(map(str, x)) + ']')

### 6. 가장 높은 확률을 가진 확률의 index 추출 후, num_to_label 로 pred_label 칼럼 변경

In [13]:
ens_result['pred_label'] = ens_result['probs_float'].apply(lambda x:num_to_label[np.argmax(x)])

In [14]:
diff_with_ens1 = ens_result['pred_label'] != ens1['pred_label']
diff_with_ens2 = ens_result['pred_label'] != ens2['pred_label']
diff_ens1_count = diff_with_ens1.sum()
diff_ens2_count = diff_with_ens2.sum()
print(f"ens1 기준 달라진 예측 개수 : {diff_ens1_count}")
print(f"ens2 기준 달라진 예측 개수 : {diff_ens2_count}")

ens1 기준 달라진 예측 개수 : 429
ens2 기준 달라진 예측 개수 : 292


In [15]:
# 형식 맞추기 위해 probs_float 드롭
ens_result = ens_result.drop(columns=['probs_float'])

### 7. 제출 가능한 형태의 csv로 출력

In [16]:
ens_result.to_csv(f'../code/prediction/ens_{ens1_name}_{ens2_name}.csv', index=False)